# Ejercicio Formativo 2 Capítulo 1
diego.herrerag00@uc.cl

In [19]:
import json

In [20]:
with open("movies.json", encoding="utf8") as movies_file:
    peliculas = json.load(movies_file)

## Analizando movies y su contenido

In [21]:
print(f"El tipo de movies es {type(peliculas)}")
print(f"Peliculas tiene {len(peliculas)} elementos")

El tipo de movies es <class 'list'>
Peliculas tiene 28795 elementos


In [22]:
print(f"El primer elemento de peliculas es {peliculas[0]}")
print(f"El primer elemento es del tipo {type(peliculas[0])}")

El primer elemento de peliculas es {'title': 'After Dark in Central Park', 'year': 1900, 'cast': [], 'genres': []}
El primer elemento es del tipo <class 'dict'>


Entonces movies es una lista de diccionarios, donde cada diccionario representa una película. Cada película tiene un título, un año de lanzamiento, una lista de actores (cast) y una lista de géneros (genres).

Entonces, para acceder a la información de una película, primero accedemos a la lista de películas y luego al índice de la película que queremos ver. Por ejemplo, para acceder al título de la primera película, usamos `peliculas[0]['title']` y se haría lo mismo para acceder a los otros datos

# Misión 1: Modelación de entidades

In [23]:
class Actor:
    def __init__(self, nombre_completo, inicio_carrera):
        self.nombre_completo = nombre_completo
        self.n_peliculas = 0
        self.inicio_carrera = inicio_carrera
        self.fin_carrera = inicio_carrera

    def actualizar_historial(self, año):
        if self.inicio_carrera > año:
            self.inicio_carrera = año
        if self.fin_carrera < año:
            self.fin_carrera = año

    def años_historial(self):
        return self.fin_carrera - self.inicio_carrera

    def __repr__(self):
        return f"{self.nombre_completo}"

class Genero:
    def __init__(self, nombre):
        self.nombre = nombre
        self.n_peliculas = 0

    def __repr__(self):
        return f"{self.nombre}"

class Pelicula:
    def __init__(self, titulo, año, reparto, generos):
        self.titulo = titulo
        self.año = año
        self.reparto = reparto
        self.generos = generos
        self.add_info() #Notar que es posible que en un metodo se llame a otro metodo de la misma clase, 
                        #para el cual, se puede lograr usando self y con ello, como lo muestra este metodo en especifico
                        #es posible se actualizar la información, en este caso, del numero de peliculas de los actores
                        #que esta almacenado en self.reparto, atributo que viene siendo una lista de objetos de
                        #la clase "Genero"

    def add_info(self):
        for actor in self.reparto:
            actor.n_peliculas += 1
        for genero in self.generos:
            genero.n_peliculas += 1

    def __repr__(self):
        return f"{self.titulo} ({self.año}) - {self.reparto} - {self.generos}"

## Misión 2: Cargando datos a objetos

Se decidió que tanto para los actores, los géneros y las películas se utilizarían diccionarios para guardar los objetos de cada tipo.

In [24]:
dict_actores = {}
dict_generos = {}
dict_peliculas = {}

Se recorre la lista de películas y se crea un objeto de las clases correspondientes para cada actor, género y película. Luego se agrega cada uno de ellos a su respectivo diccionario.

In [25]:
for pelicula in peliculas:
    cast = []
    generos = []
    for actor in pelicula["cast"]:
        if actor not in dict_actores:
            dict_actores[actor] = Actor(actor, pelicula["year"])
        else:
            dict_actores[actor].actualizar_historial(pelicula["year"])
        cast.append(dict_actores[actor])
    for genero in pelicula["genres"]:
        if genero not in dict_generos:
            dict_generos[genero] = Genero(genero)
        generos.append(dict_generos[genero])
    if (pelicula["title"], pelicula["year"],tuple(cast)) not in dict_peliculas:
        dict_peliculas[(pelicula["title"], pelicula["year"], tuple(cast))] = Pelicula(pelicula["title"], pelicula["year"], tuple(cast), generos)
    else:
        print("*"*80)
        print(f"Película duplicada: {pelicula['title']} - ({pelicula['year']}) - {tuple(cast)} - {generos}")
        pelicula_found = dict_peliculas[(pelicula["title"], pelicula["year"], tuple(cast))]
        print(f"En el diccionario ya está: {pelicula_found}")
        print("*"*80)


********************************************************************************
Película duplicada: The Prince and Betty - (1919) - (William Desmond, Mary Thurman) - [Comedy]
En el diccionario ya está: The Prince and Betty (1919) - (William Desmond, Mary Thurman) - [Comedy]
********************************************************************************
********************************************************************************
Película duplicada: The Virtuous Thief - (1919) - (Enid Bennett, Niles Welch) - [Drama]
En el diccionario ya está: The Virtuous Thief (1919) - (Enid Bennett, Niles Welch) - [Drama]
********************************************************************************
********************************************************************************
Película duplicada: The Wise Kid - (1922) - (Gladys Walton, David Butler) - [Comedy]
En el diccionario ya está: The Wise Kid (1922) - (Gladys Walton, David Butler) - [Comedy]
********************************************

Para almacenar los objetos de las películas se utiliza un diccionario, donde la llave es una tupla que contiene el título de la película, el año de lanzamiento y una tupla con los nombres de los actores. Dicho esto se encontraron películas con el mismo título, año de lanzamiento y actores, para evitar tener dos objetos de la misma película se decidió utilizar un `if` para verificar si la película ya estaba en el diccionario de películas.

In [26]:
print(f"dict_peliculas tiene {len(dict_peliculas)} elementos")

dict_peliculas tiene 28789 elementos


Se puede observar que el diccionario tiene almacenados 28789 elementos, lo cual es el número de películas que se encuentran en `peliculas`, del archivo json, sin contar las 6 repeticiones. Por ende la creación de objetos de las películas se realizó de manera correcta.

# Misión 3: Consultas sobre los datos

### Encuentre los 5 géneros más populares

In [27]:
def generos_populares(dict_generos):
    lista_generos = list(dict_generos.values())
    for i in range(len(lista_generos)):
        for j in range(i + 1, len(lista_generos)):
            if lista_generos[j].n_peliculas > lista_generos[i].n_peliculas:
                lista_generos[i], lista_generos[j] = lista_generos[j], lista_generos[i]
                
    generos_mas_populares = lista_generos[:5]
    print("Los 5 géneros más populares son:")
    for genero in generos_mas_populares:
        print(f"{genero.nombre} hay {genero.n_peliculas} películas de este género")

In [28]:
generos_populares(dict_generos)

Los 5 géneros más populares son:
Drama hay 8742 películas de este género
Comedy hay 7361 películas de este género
Western hay 3011 películas de este género
Crime hay 1499 películas de este género
Horror hay 1166 películas de este género


### Encuentre los 3 años con más películas estrenadas.

In [29]:
def peliculas_mas_premiadas(dict_peliculas):
    dict_premios = {}
    for movie in dict_peliculas.values():
        if movie.año not in dict_premios:
            dict_premios[movie.año] = 0
        dict_premios[movie.año] += 1

    lista_años = list(dict_premios.items())

    for i in range(len(lista_años) - 1):
        for j in range(i + 1, len(lista_años)):
            if lista_años[j][1] > lista_años[i][1]:
                lista_años[i], lista_años[j] = lista_años[j], lista_años[i]

    lista_años = lista_años[:3]
    
    print("Los 3 años con más películas estrenadas son:")
    for data in lista_años:
        print(f"El año {data[0]} con {data[1]} películas producidas")


In [30]:
peliculas_mas_premiadas(dict_peliculas)

Los 3 años con más películas estrenadas son:
El año 1919 con 632 películas producidas
El año 1925 con 572 películas producidas
El año 1936 con 504 películas producidas


### Encuentre a los 5 actores con la trayectoria más larga, es decir, mayor cantidad de años actuando

In [31]:
def trayectorias_mas_largas(dict_actores):
    lista_actores = list(dict_actores.values())
    for i in range(len(lista_actores)):
        for j in range(i + 1, len(lista_actores)):
            if lista_actores[j].años_historial() > lista_actores[i].años_historial():
                lista_actores[i], lista_actores[j] = lista_actores[j], lista_actores[i]
    lista_actores = lista_actores[:5]
    print("Los 5 actores con trayectoria más larga son:")
    for actor in lista_actores:
        print(f"{actor.nombre_completo} con {actor.años_historial()} años de trayectoria")

In [32]:
trayectorias_mas_largas(dict_actores)

Los 5 actores con trayectoria más larga son:
. con 101 años de trayectoria
and con 98 años de trayectoria
Harrison Ford con 98 años de trayectoria
Gloria Stuart con 80 años de trayectoria
Lillian Gish con 75 años de trayectoria


### Encuentre el reparto de una película (2 o más actores) que se haya repetido completo en otras la mayor cantidad de veces.

In [33]:
def cast_mas_repetidos(dict_peliculas):
    dict_cast = {}
    for pelicula in dict_peliculas.values():
        if len(pelicula.reparto) >= 2:
            if pelicula.reparto not in dict_cast:
                dict_cast[pelicula.reparto] = 0
            dict_cast[pelicula.reparto] += 1
        else:
            continue    
    
    max_repeticiones = -1
    max_cast = None
    for cast, repeticiones in dict_cast.items():
        if repeticiones > max_repeticiones:
            max_repeticiones = repeticiones
            max_cast = cast
    print(f"El reparto de una película que más se ha repetido es {max_cast} con {max_repeticiones} repeticiones")

In [34]:
cast_mas_repetidos(dict_peliculas)

El reparto de una película que más se ha repetido es (Harold Lloyd, Bebe Daniels) con 44 repeticiones


# Test Case

### Test de Clases

In [35]:
import unittest #Librería que nos sirve para hacer los test case

class TestE2Clases(unittest.TestCase): #Debe heredar de la clase TestCase de la libreria de unittest.
    def test_actor_historial(self):
        actor = Actor("Tom Hanks", 1990)
        actor.actualizar_historial(1985)
        actor.actualizar_historial(2000)
        self.assertEqual(actor.inicio_carrera, 1985) #assertEqual(a, b) verifica que a == b. En caso de que se cumpla, nos avisa con un "ok"
                                                     #En caso de que no se cumpla, nos avisa con un "FAIL"
        self.assertEqual(actor.fin_carrera, 2000)
        self.assertEqual(actor.años_historial(), 15)

    def test_genero_repr(self):
        genero = Genero("Drama")
        self.assertEqual(repr(genero), "Drama")
        genero.n_peliculas += 1
        self.assertEqual(genero.n_peliculas, 1)

    def test_pelicula_agregar_info(self):
        actor1 = Actor("Actor Uno", 2000)
        actor2 = Actor("Actor Dos", 2005)
        genero1 = Genero("Comedia")
        genero2 = Genero("Acción")
        pelicula = Pelicula("Mi Película", 2010, (actor1, actor2), [genero1, genero2])
        self.assertEqual(actor1.n_peliculas, 1)
        self.assertEqual(actor2.n_peliculas, 1)
        self.assertEqual(genero1.n_peliculas, 1)
        self.assertEqual(genero2.n_peliculas, 1)
        self.assertIn("Mi Película", repr(pelicula))


if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2) # Lo que va en parentesis es útil para cuando se tengan test en Jupyter notebooks
                                                                          # o en archivos de extensión .ipynb, para evitar problemas con el kernel.



test_actor_historial (__main__.TestE2Clases.test_actor_historial) ... ok
test_genero_repr (__main__.TestE2Clases.test_genero_repr) ... ok
test_pelicula_agregar_info (__main__.TestE2Clases.test_pelicula_agregar_info) ... ok
test_cast_mas_repetidos (__main__.TestFuncionesE2.test_cast_mas_repetidos) ... ok
test_generos_populares (__main__.TestFuncionesE2.test_generos_populares) ... ok
test_peliculas_mas_premiadas (__main__.TestFuncionesE2.test_peliculas_mas_premiadas) ... ok
test_trayectorias_mas_largas (__main__.TestFuncionesE2.test_trayectorias_mas_largas) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.009s

OK


El reparto de una película que más se ha repetido es (Actor Uno, Actor Dos) con 1 repeticiones
Los 5 géneros más populares son:
Comedia hay 2 películas de este género
Acción hay 1 películas de este género
Los 3 años con más películas estrenadas son:
El año 2010 con 1 películas producidas
El año 2012 con 1 películas producidas
Los 5 actores con trayectoria más larga son:
Actor Uno con 0 años de trayectoria
Actor Dos con 0 años de trayectoria


### Test de Funciones

In [36]:
class TestFuncionesE2(unittest.TestCase):
    def setUp(self):
        # Esto es de prueba, creamos los actores y géneros para luego probarlos.
        self.actor1 = Actor("Actor Uno", 2000)
        self.actor2 = Actor("Actor Dos", 2005)
        self.genero1 = Genero("Comedia")
        self.genero2 = Genero("Acción")
        self.pelicula1 = Pelicula("Mi Película", 2010, (self.actor1, self.actor2), [self.genero1, self.genero2])
        self.pelicula2 = Pelicula("Otra Película", 2012, (self.actor1,), [self.genero1])
        self.dict_actores = {self.actor1.nombre_completo: self.actor1, self.actor2.nombre_completo: self.actor2}
        self.dict_generos = {self.genero1.nombre: self.genero1, self.genero2.nombre: self.genero2}
        self.dict_peliculas = {
            ("Mi Película", 2010, (self.actor1, self.actor2)): self.pelicula1,
            ("Otra Película", 2012, (self.actor1,)): self.pelicula2
        }

    def test_generos_populares(self):
        # Solo verificamos que la función no arroje error
        generos_populares(self.dict_generos)

    def test_peliculas_mas_premiadas(self):
        peliculas_mas_premiadas(self.dict_peliculas)

    def test_trayectorias_mas_largas(self):
        trayectorias_mas_largas(self.dict_actores)

    def test_cast_mas_repetidos(self):
        cast_mas_repetidos(self.dict_peliculas)

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2)

test_actor_historial (__main__.TestE2Clases.test_actor_historial) ... ok
test_genero_repr (__main__.TestE2Clases.test_genero_repr) ... ok
test_pelicula_agregar_info (__main__.TestE2Clases.test_pelicula_agregar_info) ... ok
test_cast_mas_repetidos (__main__.TestFuncionesE2.test_cast_mas_repetidos) ... ok
test_generos_populares (__main__.TestFuncionesE2.test_generos_populares) ... ok
test_peliculas_mas_premiadas (__main__.TestFuncionesE2.test_peliculas_mas_premiadas) ... ok
test_trayectorias_mas_largas (__main__.TestFuncionesE2.test_trayectorias_mas_largas) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.003s

OK


El reparto de una película que más se ha repetido es (Actor Uno, Actor Dos) con 1 repeticiones
Los 5 géneros más populares son:
Comedia hay 2 películas de este género
Acción hay 1 películas de este género
Los 3 años con más películas estrenadas son:
El año 2010 con 1 películas producidas
El año 2012 con 1 películas producidas
Los 5 actores con trayectoria más larga son:
Actor Uno con 0 años de trayectoria
Actor Dos con 0 años de trayectoria
